In [ ]:
%load_ext autoreload
%aimport rasterio_helpers
%aimport proj_scope_vars
%autoreload 1
%autosave 300

import importlib 

import pdal
import glob
import json
from rasterio_helpers import *
from proj_scope_vars import *
import rasterio
from rasterio.warp import reproject
import numpy as np

from arosics import DESHIFTER
from arosics import COREG_LOCAL
from geoarray import GeoArray
import os, shutil, glob
import numpy as np
import time
import re
from shapely.geometry import shape
import fiona


In [ ]:
ps_sf_tiffs=glob.glob(pscope_intraband_aligned_dir + '/*.tif')
ps_sf_ref_tiff  = ps_sf_tiffs[0]

ps_arosic_args = {
    'grid_res'     : 100,
    'min_reliability' : 40,
    'window_size'  : (256,256)
}

for ps_sf_tgt_tiff in ps_sf_tiffs:
    base_fn = os.path.splitext(os.path.basename(ps_sf_tgt_tiff))[0].replace('_iba', '')
    out_tiff =  f'{aligned_raster_dir}/{base_fn}.tif' 
    if ps_sf_tgt_tiff == ps_sf_ref_tiff:
        shutil.copyfile(ps_sf_tgt_tiff, out_tiff)
    else:
        align_rasters(ps_sf_ref_tiff, ps_sf_tgt_tiff, out_tiff, arosic_args=ps_arosic_args)

sks_arosic_args = {
    'grid_res'     : 700,
    'min_reliability' : 30,
    'window_size'  : (256,256)
}

sks_pan_tiff     = f'{skysat_processed_dir}/{skysat_basename}_pan_merged_masked.tif'

ps_sf_ref_masked_tiff = f'{temp_dir}/{os.path.basename(ps_sf_ref_tiff)}'
ps_sf_ref_ds = rasterio.open(ps_sf_ref_tiff)
save_ds(mask_ds(ps_sf_ref_ds, training_area_mask, nodata=0.0), ps_sf_ref_masked_tiff)

sks_sf_pan_aligned_tiff =  f'{aligned_raster_dir}/{skysat_basename}_pan_aligned.tif' 
CRL = align_rasters(ps_sf_ref_masked_tiff, sks_pan_tiff,  sks_sf_pan_aligned_tiff, arosic_args=sks_arosic_args)
os.remove(ps_sf_ref_masked_tiff)

sks_ms_tiff     = f'{skysat_processed_dir}/{skysat_basename}_ms_merged_masked.tif'
sks_sf_ms_aligned_tiff =  f'{aligned_raster_dir}/{skysat_basename}_ms_aligned.tif' 

sks_deshift_args = {
    'path_out'     : sks_sf_ms_aligned_tiff,
    'fmt_out'      : 'GTIFF',
    'CPUs'         : 6,
    'nodata'       : 0.0,
}

DESHIFTER(sks_ms_tiff, CRL.coreg_info, **sks_deshift_args).correct_shifts()

height_map_tiffs = [height_map_dl_tif, height_map_max_tif, height_map_tif]

ld_arosic_args = {
    'grid_res'     : 350,
    'window_size'  : (1024,1024),
    'nodata'       : (0,-9999),
    'min_reliability' : 0,
}

for height_map_tiff in height_map_tiffs:
    height_map_tiff_ds = rasterio.open(height_map_tiff)
    height_map_masked_tiff = f'{temp_dir}/{os.path.basename(height_map_tiff)}'
    save_ds(mask_ds(height_map_tiff_ds, training_area_mask, nodata=-9999), height_map_masked_tiff)
    base_fn = os.path.splitext(os.path.basename(height_map_tiff))[0]
    height_map_aligned_tiff =  f'{aligned_raster_dir}/{base_fn}_aligned.tif' 
    
    print(f'height_map_masked_tiff: {height_map_masked_tiff}')
    print(f'height_map_aligned_tiff: {height_map_aligned_tiff}')
    
    if  height_map_tiff == height_map_tiffs[0]:
        CRL = align_rasters(sks_sf_pan_aligned_tiff, 
                            height_map_masked_tiff, 
                            height_map_aligned_tiff, 
                            arosic_args=ld_arosic_args)
    else:

        ld_deshift_args = {
            'path_out'     : height_map_aligned_tiff,
            'fmt_out'      : 'GTIFF',
            'CPUs'         : 6,
            'nodata'       : -9999,
        }

        DESHIFTER(height_map_masked_tiff, CRL.coreg_info, **ld_deshift_args).correct_shifts()
    
    os.remove(height_map_masked_tiff)


In [ ]:
height_map_tiffs = [height_map_dl_tif, height_map_max_tif, height_map_tif]

ld_arosic_args = {
    'grid_res'     : 350,
    'window_size'  : (1024,1024),
    'nodata'       : (0,-9999),
    'min_reliability' : 0,
}

for height_map_tiff in height_map_tiffs:
    height_map_tiff_ds = rasterio.open(height_map_tiff)
    height_map_masked_tiff = f'{temp_dir}/{os.path.basename(height_map_tiff)}'
    save_ds(mask_ds(height_map_tiff_ds, training_area_mask, nodata=-9999), height_map_masked_tiff)
    base_fn = os.path.splitext(os.path.basename(height_map_tiff))[0]
    height_map_aligned_tiff =  f'{aligned_raster_dir}/{base_fn}_aligned.tif' 
    
    print(f'height_map_masked_tiff: {height_map_masked_tiff}')
    print(f'height_map_aligned_tiff: {height_map_aligned_tiff}')
    
    if  height_map_tiff == height_map_tiffs[0]:
        CRL = align_rasters(sks_sf_pan_aligned_tiff, 
                            height_map_masked_tiff, 
                            height_map_aligned_tiff, 
                            arosic_args=ld_arosic_args)
    else:

        ld_deshift_args = {
            'path_out'     : height_map_aligned_tiff,
            'fmt_out'      : 'GTIFF',
            'CPUs'         : 6,
            'nodata'       : -9999,
        }

        DESHIFTER(height_map_masked_tiff, CRL.coreg_info, **ld_deshift_args).correct_shifts()
    
    os.remove(height_map_masked_tiff)